# Conexion Base de Datos Postgres

In [20]:
import psycopg
import os

# Define tus parámetros de conexión
conn_params = {
    'dbname': 'datos',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432'
}

# Establece la conexión
try:
    conn = psycopg.connect(**conn_params)
    print("Conexión exitosa")
except Exception as e:
    print(f"Ocurrió un error: {e}")

# No olvides cerrar la conexión cuando termines
# conn.close()

Conexión exitosa


# 1. Función que elimine los espacios en blanco de la columna distrito para usarse en ambos conjuntos de datos.

In [21]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import regexp_replace 
 
# Crea una sesión de Spark 
spark = SparkSession.builder \
    .appName("Preprocessing Data") \
    .getOrCreate() 
 
# Carga datos desde los archivos CSV 
oij_df = spark.read.csv("C:\\Users\\andco\\Desktop\\spark\\data\\OIJ.csv", header=True, inferSchema=True) 
inec_df = spark.read.csv("C:\\Users\\andco\\Desktop\\spark\\data\\INEC.csv", header=True, inferSchema=True) 
 
def eliminar_espacios_y_concatenar(dataframe, columna): 
    """ 
    Elimina los espacios en blanco y concatena las palabras en una columna específica de un DataFrame de Spark. 
    """ 
    if columna in dataframe.columns: 
        return dataframe.withColumn(columna, regexp_replace(dataframe[columna], " ", "")) 
    else: 
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.") 
 
# Limpia la columna 'Distrito' en el DataFrame de OIJ 
try: 
    oij_df = eliminar_espacios_y_concatenar(oij_df, 'Distrito') 
    # Elimina la columna '_C11' si existe 
    if '_C11' in oij_df.columns: 
        oij_df = oij_df.drop('_C11') 
except Exception as e: 
    print(f"Error al procesar OIJ: {e}") 
 
# Limpia la columna 'distrito' en el DataFrame de INEC 
try: 
    inec_df = eliminar_espacios_y_concatenar(inec_df, 'distrito') 
except Exception as e: 
    print(f"Error al procesar INEC: {e}") 
 
# Muestra los resultados 
print("Datos del OIJ después de limpiar:") 
oij_df.show(50, truncate=False) 
 
print("Datos del INEC después de limpiar:") 
inec_df.show(50, truncate=False)

Datos del OIJ después de limpiar:
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+--------------------+
|Delito|SubDelito  |Fecha     |Hora               |Victima |SubVictima                         |Edad         |Sexo  |Nacionalidad|Provincia |Canton       |Distrito            |
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+--------------------+
|ASALTO|ARMA BLANCA|1/10/2011 |18:00:00 - 20:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|NICARAGUA   |SAN JOSE  |ALAJUELITA   |SANFELIPE           |
|ASALTO|ARMA BLANCA|2/2/2011  |03:00:00 - 05:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|COSTA RICA  |CARTAGO   |OREAMUNO     |SANRAFAEL           |
|ASALTO|ARMA BLANCA|10/23/2011|21:00:00 - 23:59:59|PERSONA |PEATON [PERSONA]     

## 2. Función que convierta a minúsculas el contenido de la columna distrito para usarse en ambos conjuntos de datos

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower

# Crear SparkSession
try:
    spark = SparkSession.builder \
        .appName("Proyecto SparkSQL") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.7.4") \
        .getOrCreate()
    print("SparkSession creada:", spark.version)
except Exception as e:
    print("Error al crear SparkSession:", str(e))

# Cargar CSVs
#df_oij = spark.read.csv("C:\\Projects\\spark\\data\\OIJ2011.csv", header=True, inferSchema=True)
#df_inec = spark.read.csv("C:\\Projects\\spark\\data\\inec.csv", header=True, inferSchema=True)

# Función para convertir a minúsculas
def convertir_minusculas(df, columna):
    """
    Convierte los valores de una columna a minúsculas.
    
    Args:
        df: DataFrame de Spark.
        columna: Nombre de la columna a transformar.
        
    Returns:
        DataFrame con los valores en minúsculas.
    """
    return df.withColumn(columna, lower(df[columna]))

# Aplicar la función en ambos DataFrames
df_oij = convertir_minusculas(oij_df, 'Distrito')
df_inec = convertir_minusculas(inec_df, 'Distrito')

# Mostrar resultados
print("Datos OIJ:")
df_oij.show(truncate=False)

print("Datos INEC:")
df_inec.show(truncate=False)

SparkSession creada: 3.5.3
Datos OIJ:
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+--------------------+
|Delito|SubDelito  |Fecha     |Hora               |Victima |SubVictima                         |Edad         |Sexo  |Nacionalidad|Provincia |Canton       |Distrito            |
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+--------------------+
|ASALTO|ARMA BLANCA|1/10/2011 |18:00:00 - 20:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|NICARAGUA   |SAN JOSE  |ALAJUELITA   |sanfelipe           |
|ASALTO|ARMA BLANCA|2/2/2011  |03:00:00 - 05:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|COSTA RICA  |CARTAGO   |OREAMUNO     |sanrafael           |
|ASALTO|ARMA BLANCA|10/23/2011|21:00:00 - 23:59:59|PERSONA |PEATON [PERSONA] 

# Crear Tablas para Ambos DataFrames

In [24]:
import psycopg
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, BooleanType

# Función para generar script SQL para crear tabla
def crear_tabla_sql(df, nombre_tabla):
    """
    Genera el script SQL para crear una tabla en PostgreSQL desde un DataFrame.
    """
    schema = df.schema
    columnas = []
    for campo in schema.fields:
        if isinstance(campo.dataType, StringType):
            tipo_postgres = "VARCHAR"
        elif isinstance(campo.dataType, IntegerType):
            tipo_postgres = "INTEGER"
        elif isinstance(campo.dataType, FloatType):
            tipo_postgres = "FLOAT"
        elif isinstance(campo.dataType, DoubleType):
            tipo_postgres = "DOUBLE PRECISION"
        elif isinstance(campo.dataType, BooleanType):
            tipo_postgres = "BOOLEAN"
        else:
            tipo_postgres = "TEXT"
        columnas.append(f"{campo.name} {tipo_postgres}")
    
    columnas_str = ", ".join(columnas)
    return f"CREATE TABLE IF NOT EXISTS {nombre_tabla} ({columnas_str});"

# Función para insertar datos en PostgreSQL
def insertar_datos_postgres(df, nombre_tabla, conn_params):
    """
    Inserta los datos de un DataFrame en una tabla PostgreSQL.
    
    Args:
        df: DataFrame de Spark.
        nombre_tabla: Nombre de la tabla en PostgreSQL.
        conn_params: Parámetros de conexión.
    """
    datos = [tuple(row) for row in df.collect()]  # Convierte el DataFrame en una lista de tuplas
    columnas = ", ".join(df.columns)
    placeholders = ", ".join(["%s"] * len(df.columns))
    sql_insertar = f"INSERT INTO {nombre_tabla} ({columnas}) VALUES ({placeholders})"
    
    try:
        with psycopg.connect(**conn_params) as conn:
            with conn.cursor() as cur:
                cur.executemany(sql_insertar, datos)
            conn.commit()
        print(f"Datos insertados correctamente en la tabla '{nombre_tabla}'.")
    except Exception as e:
        print(f"Error al insertar datos en la tabla '{nombre_tabla}': {e}")

# Ajustar columnas para reflejar los cambios en el archivo INEC
df_oij = df_oij.toDF(*[
    "delito", "subdelito", "fecha", "hora", "victima", "subvictima",
    "edad", "sexo", "nacionalidad", "provincia", "canton", "distrito"
])  # Renombrar columnas para PostgreSQL

df_inec = df_inec.toDF(
    "provincia", "canton", "distrito", "poblacion_mayor_a_15",
    "tasa_neta_de_participacion", "tasa_de_ocupacion",
    "tasa_de_desempleo_abierto", "porcentaje_de_poblacion_economicamente_inactiva",
    "relacion_de_dependencia_economica", "porcentaje_poblacion_sector_primario",
    "porcentaje_poblacion_sector_secundario", "porcentaje_poblacion_sector_terciario"
)

# Crear tablas y columnas con nombres ajustados
tablas = [
    ("oij", df_oij),
    ("inec", df_inec)
]

for nombre_tabla, dataframe in tablas:
    try:
        # Ajustar nombres de columnas para evitar errores
        dataframe = dataframe.toDF(*[col.replace(" ", "_").lower() for col in dataframe.columns])

        # Crear tabla
        sql_crear_tabla = crear_tabla_sql(dataframe, nombre_tabla)
        with psycopg.connect(**conn_params) as conn:
            with conn.cursor() as cur:
                cur.execute(sql_crear_tabla)
            conn.commit()
        print(f"Tabla '{nombre_tabla}' creada correctamente.")

        # Insertar datos
        insertar_datos_postgres(dataframe, nombre_tabla, conn_params)
    except Exception as e:
        print(f"Error al procesar la tabla '{nombre_tabla}': {e}")

Tabla 'oij' creada correctamente.
Datos insertados correctamente en la tabla 'oij'.
Tabla 'inec' creada correctamente.
Datos insertados correctamente en la tabla 'inec'.


# Quitar Tildes de la Columna distritos de inec

In [26]:
from pyspark.sql.functions import translate

# Quitar tildes de la columna 'Distrito'
df_inec = df_inec.withColumn(
    "Distrito",
    translate(
        "Distrito",
        "áéíóúÁÉÍÓÚ",
        "aeiouAEIOU"
    )
)

# Mostrar resultados para verificar
df_inec.select("Distrito").show(truncate=False)


+---------------------+
|Distrito             |
+---------------------+
|carmen               |
|merced               |
|hospital             |
|catedral             |
|zapote               |
|sanfranciscodedosrios|
|uruca                |
|mataredonda          |
|pavas                |
|hatillo              |
|sansebastian         |
|escazu               |
|sanantonio           |
|sanrafael            |
|desamparados         |
|sanmiguel            |
|sanjuandedios        |
|sanrafaelarriba      |
|sanantonio           |
|frailes              |
+---------------------+
only showing top 20 rows



# Quitar tildes en la base de datos (columna distritos, tabla inec)

In [27]:
sql_update = """
UPDATE inec
SET Distrito = translate(Distrito, 'áéíóúÁÉÍÓÚ', 'aeiouAEIOU');
"""

with psycopg.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        cur.execute(sql_update)
    conn.commit()
print("Tildes eliminadas en la tabla 'inec'.")


Tildes eliminadas en la tabla 'inec'.


# 3. Una función que devuelva la lista de distritos del conjunto de datos del OIJ que no coinciden con ningún distrito del conjunto de datos del INEC

In [29]:
from pyspark.sql.functions import col, translate

def obtener_distritos_no_coincidentes(oij_df, inec_df, columna_oij, columna_inec):
    """
    Devuelve la lista de distritos del conjunto de datos del OIJ que no coinciden
    con ningún distrito del conjunto de datos del INEC.

    Args:
        oij_df: DataFrame de PySpark con los datos del OIJ.
        inec_df: DataFrame de PySpark con los datos del INEC.
        columna_oij: Nombre de la columna de distrito en el DataFrame OIJ.
        columna_inec: Nombre de la columna de distrito en el DataFrame INEC.

    Returns:
        DataFrame con los distritos del OIJ que no coinciden con los del INEC.
    """
    # Limpiar tildes en ambas columnas para asegurar comparaciones consistentes
    oij_df = oij_df.withColumn(
        columna_oij,
        translate(col(columna_oij), "áéíóúÁÉÍÓÚ", "aeiouAEIOU")
    )
    inec_df = inec_df.withColumn(
        columna_inec,
        translate(col(columna_inec), "áéíóúÁÉÍÓÚ", "aeiouAEIOU")
    )

    # Realizar el left anti join
    distritos_no_coincidentes = oij_df.join(
        inec_df,
        oij_df[columna_oij] == inec_df[columna_inec],
        how="left_anti"
    )

    # Seleccionar únicamente la columna de distritos del OIJ
    return distritos_no_coincidentes.select(columna_oij).distinct()

# Uso de la función
distritos_no_coincidentes = obtener_distritos_no_coincidentes(
    oij_df=df_oij,
    inec_df=df_inec,
    columna_oij="distrito",
    columna_inec="Distrito"
)

# Mostrar los distritos no coincidentes
distritos_no_coincidentes.show(truncate=False)

+------------------+
|distrito          |
+------------------+
|canonegro         |
|lalegua           |
|granja            |
|canasdulces       |
|desconocido       |
|cortes            |
|palmera           |
|sanjosedelamonta?a|
|general           |
|horquetas         |
|ceiba             |
|elchirripo        |
|mastate           |
|fortuna           |
|mataplatano       |
|losangeles        |
|tapezco           |
|penasblancas      |
|merecedes         |
|canas             |
+------------------+
only showing top 20 rows



# 4. Una función que devuelva la cantidad de registros en el conjunto de datos del OIJ que no coinciden con ningún distrito del conjunto de datos del INEC.

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, col, translate

def contar_distritos_no_coincidentes(oij_df, inec_df, columna_oij="Distrito", columna_inec="Distrito"):
    """
    Cuenta la cantidad de registros en el conjunto de datos del OIJ 
    que no coinciden con ningún distrito del conjunto de datos del INEC.

    Args:
        oij_df: DataFrame del conjunto de datos del OIJ.
        inec_df: DataFrame del conjunto de datos del INEC.
        columna_oij (str): Nombre de la columna de distrito en el DataFrame OIJ.
        columna_inec (str): Nombre de la columna de distrito en el DataFrame INEC.

    Returns:
        int: Número de registros del OIJ sin coincidencia en INEC.
    """
    # Normalizar la columna de distrito en OIJ eliminando tildes, espacios y pasando a minúsculas
    oij_df = oij_df.withColumn(
        columna_oij,
        translate(trim(lower(col(columna_oij))), "áéíóúÁÉÍÓÚ", "aeiouAEIOU")
    )
    
    # Normalizar la columna de distrito en INEC eliminando tildes, espacios y pasando a minúsculas
    inec_df = inec_df.withColumn(
        columna_inec,
        translate(trim(lower(col(columna_inec))), "áéíóúÁÉÍÓÚ", "aeiouAEIOU")
    )
    
    # Realizar un left anti join para encontrar los distritos del OIJ que no tienen coincidencia en INEC
    distritos_no_coincidentes = oij_df.join(
        inec_df,
        oij_df[columna_oij] == inec_df[columna_inec],
        how="left_anti"
    )
    
    # Contar y devolver el número de registros no coincidentes
    cantidad_no_coincidentes = distritos_no_coincidentes.count()
    return cantidad_no_coincidentes

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Comparar Distritos OIJ e INEC") \
    .getOrCreate()

# Cargar los datasets desde archivos CSV con las rutas ajustadas
#df_oij = spark.read.csv("C:\\Projects\\spark\\data\\OIJ2011.csv", header=True, inferSchema=True)
#df_inec = spark.read.csv("C:\\Projects\\spark\\data\\inec.csv", header=True, inferSchema=True)

# Llamar a la función
resultado = contar_distritos_no_coincidentes(oij_df, inec_df)
print(f"Número de registros no coincidentes: {resultado}")

Número de registros no coincidentes: 1912


# Edite, utilizando SparkSQL, los nombres de los distritos del INEC para que coincidan con algunos de los del OIJ.

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, lower, col, translate

def editar_distritos_inec(oij_df, inec_df, columna_oij="Distrito", columna_inec="distrito"):
    """
    Edita los nombres de los distritos en el INEC para que coincidan con algunos de los del OIJ.

    Args:
        oij_df: DataFrame del conjunto de datos del OIJ.
        inec_df: DataFrame del conjunto de datos del INEC.
        columna_oij (str): Nombre de la columna de distrito en el DataFrame OIJ.
        columna_inec (str): Nombre de la columna de distrito en el DataFrame INEC.

    Returns:
        DataFrames editados de OIJ e INEC.
    """
    # Normalizar columnas de distritos en OIJ e INEC (quitar tildes, ñ -> n, espacios, minúsculas)
    oij_df = oij_df.withColumn(
        columna_oij,
        translate(trim(lower(col(columna_oij))), "áéíóúñÁÉÍÓÚÑ", "aeiounAEIOUN")
    )
    
    inec_df = inec_df.withColumn(
        columna_inec,
        translate(trim(lower(col(columna_inec))), "áéíóúñÁÉÍÓÚÑ", "aeiounAEIOUN")
    )
    
    return oij_df, inec_df

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Editar Distritos INEC para Coincidir con OIJ") \
    .getOrCreate()

# Cargar los datasets desde archivos CSV
#df_oij = spark.read.csv("C:\\Projects\\spark\\data\\OIJ2011.csv", header=True, inferSchema=True)
#df_inec = spark.read.csv("C:\\Projects\\spark\\data\\inec.csv", header=True, inferSchema=True)

# Llamar a la función para editar distritos
df_oij_editado, df_inec_editado = editar_distritos_inec(df_oij, df_inec)

# Usar alias para desambiguar columnas
df_oij_editado = df_oij_editado.alias("oij")
df_inec_editado = df_inec_editado.alias("inec")

# Verificar coincidencias específicas para "cañas" y "peñas blancas"
coincidencias_especificas = df_oij_editado.join(
    df_inec_editado,
    col("oij.Distrito") == col("inec.distrito"),
    "inner"
).filter(
    (col("oij.Distrito").like("%canas%")) |
    (col("oij.Distrito").like("%penas blancas%"))
)

print("Coincidencias específicas para 'cañas' y 'peñas blancas':")
coincidencias_especificas.select("oij.Distrito", "inec.distrito").distinct().show(truncate=False)

Coincidencias específicas para 'cañas' y 'peñas blancas':
+-----------+-----------+
|Distrito   |distrito   |
+-----------+-----------+
|canas      |canas      |
|canasdulces|canasdulces|
+-----------+-----------+

